- ~~Total amount raised (across all products)~~
- ~~\# of one-time donations (across all products)~~
- ~~\# of recurring donations (across all products)~~
- ~~Avg. one-time gift size (across all products)~~
- ~~Median one-time gift size (across all products)~~
- Total amount raised (events, auctions, p2p)

In [1]:
import pandas as pd

import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

# all orgs

In [36]:
q = '''select
            date_trunc('quarter', date) as quarter,
            count(distinct(org)) as org_count,
            count(id) as trans_count,
            sum(amount) as trans_sum,
            avg(amount) as trans_mean,
            count(case when recurring=0 then id else null end) as onetime_count
        from transactions
        where
            status='A' and
            year>=2022 
        group by date_trunc('quarter', date)'''
df = redshift_query_read(q, schema='production')

In [37]:
q = '''select
            date_trunc('quarter', date) as quarter,
            median(amount) as trans_median
        from transactions
        where
            status='A' and
            year>=2022
        group by date_trunc('quarter', date)'''
df_mdn = redshift_query_read(q, schema='production')

In [38]:
q = '''select 
            date_trunc('quarter', date) as quarter,
            count(case when recurring_origin=1 then id else null end) as new_recurring
        from transactions
        where
            status='A' and
            year>=2022
        group by date_trunc('quarter', date)'''
df_newrec = redshift_query_read(q, schema='production')

In [39]:
q = '''select
            date_trunc('quarter', date) as quarter,
            count(id) as event_count
        from transactions
        where
            status='A' and
            year>=2022 and
            (events_count>0 or source='p2p' or auctionpurchase_count>0 or auctiondonation_count>0)
        group by date_trunc('quarter', date)'''

df_totaleventsaucp2p = redshift_query_read(q, schema='production')

In [40]:
df = df.merge(df_mdn, on='quarter').merge(df_newrec, on='quarter').merge(df_totaleventsaucp2p, on='quarter')

In [41]:
qs = ['2023-01-01', '2022-01-01']
df = df[df['quarter'].isin(qs)]
df.sort_values('quarter', ascending=True, inplace=True)

In [42]:
df[[c for c in df.columns if c!='quarter']].pct_change().transpose()

,1,4
org_count,NaN,0.102386
trans_count,NaN,0.108618
trans_sum,NaN,0.157341
trans_mean,NaN,0.043949
onetime_count,NaN,0.103202
trans_median,NaN,0.000000
new_recurring,NaN,0.014459
event_count,NaN,0.247346


In [35]:
df

,quarter,trans_count,trans_sum,trans_mean,onetime_count,trans_median_x,new_recurring_x,event_count_x,trans_median_y,new_recurring_y,event_count_y
0,2022-01-01,650879,6.949724e+07,106.774442,352010,30.0,14662,186605,30.0,14662,186605
1,2023-01-01,721576,8.043200e+07,111.467119,388338,30.0,14874,232761,30.0,14874,232761


Q1 2022, 2023 YoY

- Transaction count: 10.86%
- Total amount raised: 15.73%
- \# of one-time donations: 10.32%
- \# of recurring donations: 1.45%
- Avg. one-time: 4.39%
- Median one-time: 0.00%
- Total (events, auctions, p2p): 24.73%

# orgs active in 2022 and 2023

In [46]:
q = '''select
            org,
            date_trunc('quarter', date) as quarter,
            count(id) as trans_count,
            sum(amount) as trans_vol,
            count(case when recurring=0 then id else null end) as onetime_count,
            count(case when recurring_origin=1 then id else null end) as rec_count
        from transactions
        where
            status='A' and
            year>='2022' 
        group by date_trunc('quarter', date), org'''
org_quarter = redshift_query_read(q, schema='production')

In [48]:
quarters = ['2023-01-01', '2022-01-01']
org_quarter = org_quarter[org_quarter['quarter'].isin(quarters)]
orgs_2022 = org_quarter[org_quarter['quarter']=='2022-01-01']['org'].to_list()
org_quarter = org_quarter[org_quarter['org'].isin(orgs_2022)]

In [52]:
cols = [c for c in org_quarter.columns if c not in ['org', 'quarter']]
oqs = org_quarter.groupby('quarter')[cols].sum().reset_index()
oqs.sort_values('quarter', ascending=True)[cols].pct_change().transpose()

,0,1
trans_count,NaN,0.010011
trans_vol,NaN,0.041834
onetime_count,NaN,-0.042914
rec_count,NaN,-0.117174


Orgs active in Q1 2022 & 2023

- transaction count: 1.00%
- total volume: 4.18%
- onetime count: -4.29%
- recurring count: -11.72%

# new donors Q1 2022 vs 2023

In [61]:
q = '''select 
            org,
            email as donor,
            date_trunc('quarter', date) as quarter,
            count(id) as trans_count,
            sum(amount) as trans_vol,
            count(case when recurring=0 then id else null end) as onetime_count,
            count(case when recurring_origin=1 then id else null end) as rec_count
        from transactions
        where
            status='A' and
            year>='2021' 
        group by date_trunc('quarter', date), org, email'''
donors = redshift_query_read(q, schema='production')

In [62]:
donors.tail(3)

,org,donor,quarter,trans_count,trans_vol,onetime_count,rec_count
3978935,444995,c4721a6db92bb0832ce00e71bdfe0668bd54d839,2023-04-01,4,157.5,1,0
3978936,445213,c8ae0d63e062d26a67829d421feeb1760e01ef6a,2023-07-01,1,500.0,1,0
3978937,442307,7c7ca7d25f65274dbe6bdb04879892026b8d255a,2023-01-01,1,50.0,1,0


In [ ]:
donors_2021 = donors[donors['quarter']=='2021-01-01'][['org', 'donor']].drop_duplicates()
donors_2022 = donors[donors['quarter']=='2022-01-01'][['org', 'donor']].drop_duplicates()
